In [ ]:
# pip install accelerate
#from transformers import T5Tokenizer, T5ForConditionalGeneration
#from openai import OpenAI

import json

In [ ]:
import os
os.environ['HF_HOME']='/run/cache'

In [11]:
password=os.getenv("POSTGRES_PASSWORD")
print(password)

None


In [8]:
import psycopg2
import os
password=os.getenv("POSTGRES_PASSWORD")
print(password)

try:
    conn = psycopg2.connect(
        host="localhost",
        database="primal_health",
        user="postgres",
        password=password,
        port="7777"
    )
    print("Connection successful")
    conn.close()
except Exception as e:
    print(f"Error: {e}")

None
Error: connection to server at "localhost" (::1), port 7777 failed: fe_sendauth: no password supplied



In [ ]:
import os
import psycopg2
from psycopg2.extras import DictCursor
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

RUN_TIMEZONE_CHECK = os.getenv('RUN_TIMEZONE_CHECK', '1') == '1'

TZ_INFO = os.getenv("TZ", "Europe/Berlin")
tz = ZoneInfo(TZ_INFO)


def get_db_connection():
    return psycopg2.connect(
        host=os.getenv("POSTGRES_HOST", "postgres"),
        database=os.getenv("POSTGRES_DB", "course_assistant"),
        user=os.getenv("POSTGRES_USER", "your_username"),
        password=os.getenv("POSTGRES_PASSWORD", "your_password"),
    )


def init_db():
    conn = get_db_connection()
    try:
        with conn.cursor() as cur:
            cur.execute("DROP TABLE IF EXISTS feedback")
            cur.execute("DROP TABLE IF EXISTS conversations")

            cur.execute("""
                CREATE TABLE conversations (
                    id TEXT PRIMARY KEY,
                    question TEXT NOT NULL,
                    answer TEXT NOT NULL,
                    model_used TEXT NOT NULL,
                    response_time FLOAT NOT NULL,
                    relevance TEXT NOT NULL,
                    relevance_explanation TEXT NOT NULL,
                    prompt_tokens INTEGER NOT NULL,
                    completion_tokens INTEGER NOT NULL,
                    total_tokens INTEGER NOT NULL,
                    eval_prompt_tokens INTEGER NOT NULL,
                    eval_completion_tokens INTEGER NOT NULL,
                    eval_total_tokens INTEGER NOT NULL,
                    openai_cost FLOAT NOT NULL,
                    timestamp TIMESTAMP WITH TIME ZONE NOT NULL
                )
            """)
            cur.execute("""
                CREATE TABLE feedback (
                    id SERIAL PRIMARY KEY,
                    conversation_id TEXT REFERENCES conversations(id),
                    feedback INTEGER NOT NULL,
                    timestamp TIMESTAMP WITH TIME ZONE NOT NULL
                )
            """)
        conn.commit()
    finally:
        conn.close()


def save_conversation(conversation_id, question, answer_data, timestamp=None):
    if timestamp is None:
        timestamp = datetime.now(tz)

    conn = get_db_connection()
    try:
        with conn.cursor() as cur:
            cur.execute(
                """
                INSERT INTO conversations 
                (id, question, answer, model_used, response_time, relevance, 
                relevance_explanation, prompt_tokens, completion_tokens, total_tokens, 
                eval_prompt_tokens, eval_completion_tokens, eval_total_tokens, openai_cost, timestamp)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """,
                (
                    conversation_id,
                    question,
                    answer_data["answer"],
                    answer_data["model_used"],
                    answer_data["response_time"],
                    answer_data["relevance"],
                    answer_data["relevance_explanation"],
                    answer_data["prompt_tokens"],
                    answer_data["completion_tokens"],
                    answer_data["total_tokens"],
                    answer_data["eval_prompt_tokens"],
                    answer_data["eval_completion_tokens"],
                    answer_data["eval_total_tokens"],
                    answer_data["openai_cost"],
                    timestamp
                ),
            )
        conn.commit()
    finally:
        conn.close()


def save_feedback(conversation_id, feedback, timestamp=None):
    if timestamp is None:
        timestamp = datetime.now(tz)

    conn = get_db_connection()
    try:
        with conn.cursor() as cur:
            cur.execute(
                "INSERT INTO feedback (conversation_id, feedback, timestamp) VALUES (%s, %s, COALESCE(%s, CURRENT_TIMESTAMP))",
                (conversation_id, feedback, timestamp),
            )
        conn.commit()
    finally:
        conn.close()


def get_recent_conversations(limit=5, relevance=None):
    conn = get_db_connection()
    try:
        with conn.cursor(cursor_factory=DictCursor) as cur:
            query = """
                SELECT c.*, f.feedback
                FROM conversations c
                LEFT JOIN feedback f ON c.id = f.conversation_id
            """
            if relevance:
                query += f" WHERE c.relevance = '{relevance}'"
            query += " ORDER BY c.timestamp DESC LIMIT %s"

            cur.execute(query, (limit,))
            return cur.fetchall()
    finally:
        conn.close()


def get_feedback_stats():
    conn = get_db_connection()
    try:
        with conn.cursor(cursor_factory=DictCursor) as cur:
            cur.execute("""
                SELECT 
                    SUM(CASE WHEN feedback > 0 THEN 1 ELSE 0 END) as thumbs_up,
                    SUM(CASE WHEN feedback < 0 THEN 1 ELSE 0 END) as thumbs_down
                FROM feedback
            """)
            return cur.fetchone()
    finally:
        conn.close()


def check_timezone():
    conn = get_db_connection()
    try:
        with conn.cursor() as cur:
            cur.execute("SHOW timezone;")
            db_timezone = cur.fetchone()[0]
            print(f"Database timezone: {db_timezone}")

            cur.execute("SELECT current_timestamp;")
            db_time_utc = cur.fetchone()[0]
            print(f"Database current time (UTC): {db_time_utc}")

            db_time_local = db_time_utc.astimezone(tz)
            print(f"Database current time ({TZ_INFO}): {db_time_local}")

            py_time = datetime.now(tz)
            print(f"Python current time: {py_time}")

            # Use py_time instead of tz for insertion
            cur.execute("""
                INSERT INTO conversations 
                (id, question, answer, model_used, response_time, relevance, 
                relevance_explanation, prompt_tokens, completion_tokens, total_tokens, 
                eval_prompt_tokens, eval_completion_tokens, eval_total_tokens, openai_cost, timestamp)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                RETURNING timestamp;
            """, 
            ('test', 'test question', 'test answer', 'test model', 0.0, 0.0, 
             'test explanation', 0, 0, 0, 0, 0, 0, 0.0, py_time))

            inserted_time = cur.fetchone()[0]
            print(f"Inserted time (UTC): {inserted_time}")
            print(f"Inserted time ({TZ_INFO}): {inserted_time.astimezone(tz)}")

            cur.execute("SELECT timestamp FROM conversations WHERE id = 'test';")
            selected_time = cur.fetchone()[0]
            print(f"Selected time (UTC): {selected_time}")
            print(f"Selected time ({TZ_INFO}): {selected_time.astimezone(tz)}")

            # Clean up the test entry
            cur.execute("DELETE FROM conversations WHERE id = 'test';")
            conn.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    finally:
        conn.close()


if RUN_TIMEZONE_CHECK:
    check_timezone()

In [ ]:
import transformers
import torch

from huggingface_hub import HfApi, login
import os
api_token = os.getenv("huggingface")

# Log in with your Hugging Face API token
login(token=api_token)

model_id = "meta-llama/Meta-Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

pipeline("Hey how are you doing today?")


In [ ]:
os.chdir(r'C:\Users\user\Documents\LLM Zoomcamp\Project')
print(os.getcwd())
file_path = os.path.join(os.getcwd(), 'data\clean_data\documents.json')
file_path


In [ ]:
with open(file_path, 'r') as file:
    documents=json.load(file)
len(documents)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")




In [ ]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
llm('how is weather in london?')

In [ ]:
x=3
y=9

In [ ]:
x @ y

In [ ]:
import operator

# Define matrices as lists
A = [[1, 2], [3, 4]]
B = [[5, 6], [7, 8]]

# Matrix multiplication
result = [[sum(a * b for a, b in zip(A_row, B_col)) for B_col in zip(*B)] for A_row in A]
print(result)


In [ ]:
DATA_PATH = os.getenv("DATA_PATH", "../data/clean_data/final_data_with_IDs_new.csv")


In [ ]:
DATA_PATH

In [ ]:
import os
data_path=os.getenv('Data_Path')

In [ ]:
os.getenv

In [ ]:
os.environ['Data']='rttt'

In [ ]:
os.environ['Data']


In [ ]:
import os
#from dotenv import load_dotenv

os.environ['RUN_TIMEZONE_CHECK'] = '0'

from db import init_db

#load_dotenv()

if __name__ == "__main__":
    print("Initializing database...")
    init_db()

In [ ]:
project=os.getenv('Project_Path')
import os
current=Path(project)
new=os.path.join(current, 'notebooks')
print(new)
os.chdir(new)
print(os.getcwd())

from pathlib import Path
import time
from openai import OpenAI
import minsearch
import json
import os
import subprocess

# Load documents
file_path = current / 'data'/ 'clean_data' / 'documents.json'
print(file_path)
#file_path = os.path.join(os.getcwd(), r'data/clean_data/documents.json')
with open(file_path, 'r') as file:
    documents = json.load(file)

# Initialize the OpenAI client
client = OpenAI()

# Initialize the minsearch index
index = minsearch.Index(
    text_fields=['Chunked_Content'],
    keyword_fields=[]
)
index.fit(documents)

def search(query):
    boost = {
        'Chunked_Content': 1.9366969407339725
    }
    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )
    return results

def build_prompt(query, search_results):
    prompt_template = """
    You're a primal health adviser. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = "\n".join([f"Chunked_Content: {doc['Chunked_Content']}" for doc in search_results])
    
    return prompt_template.format(question=query, context=context).strip()

# def llm(prompt):
#     response = client.chat.completions.create(
#         model='gpt-3.5-turbo',
#         messages=[{"role": "user", "content": prompt}]
#     )
#     return response.choices[0].message.content
# def llm(prompt):
#     response = client.chat.completions.create(
#         model='gpt-4o',
#         messages=[{"role": "user", "content": prompt}]
#     )
#     return response.choices[0].message.content

def llm(prompt,model="llama3.2"):
    # Command to run Ollama with the desired model and prompt
    command = ["ollama", "run", model, prompt]

    # Run the command and capture the output
    result = subprocess.run(command, capture_output=True, text=True, encoding='utf-8')

    # Return the result
    return result.stdout

# Example usage
response = query_ollama("What is the capital of France?")
print(response)
def rag(query):
    # Check if the user is asking who created the bot
    if query.lower() in ["who created you?", "who is your creator?", "who made you?"]:
        return "I was created by W Mohamed."

    # Otherwise, proceed with normal search and LLM response
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    time.sleep(1)  # Simulating a delay for the function to work
    return answer
    


In [ ]:
query='what causes belly fat'

In [ ]:
rag(query)

In [ ]:
# 'Eating raw animal fat may not be as healing as expected, and cooking the fat before digestion might be necessary, especially tough fat like pork belly.
# Fermentation could be the oldest form of cooking, but dry aging and wet fermenting may have different microbial effects. High meat consumption may 
# not work for everyone, as it can lead to a nightmare experience and changes in taste. Ultimately, 
# the causes of belly fat may vary from person to person.'


In [ ]:
from pathlib import Path
cwd = Path.cwd()
goal_dir = cwd.parent.parent / "my_dir"

In [ ]:
cwd

In [ ]:
goal_dir

In [ ]:
cwd.parent

In [ ]:
os.path.join(os.getcwd(), data_path)

In [ ]:
path = Path(r'C:\Users\user\Documents\LLM Zoomcamp\Project\data\clean_data')

In [ ]:
path

In [ ]:
import subprocess

def query_ollama(prompt,model="llama3.2"):
    # Command to run Ollama with the desired model and prompt
    command = ["ollama", "run", model, prompt]

    # Run the command and capture the output
    result = subprocess.run(command, capture_output=True, text=True, encoding='utf-8')

    # Return the result
    return result.stdout

# Example usage
response = query_ollama("What is the capital of France?")
print(response)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set the model name to use LLaMA model (e.g., LLaMA 7B)
model_name = "decapoda-research/llama3.2:latest"  # Example: LLaMA-7B model

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Text input
input_text = "What is the capital of France?"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Generate text using the model
outputs = model.generate(**inputs, max_length=50)

# Decode and print the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
query='tell me about healthy food?'
def llm_flan(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt",max_length=2048, truncation=True).input_ids
    #outputs = model.generate(input_ids, max_length=3048, num_beams=4, early_stopping=True)
    outputs = model.generate(
        input_ids, 
        max_new_tokens=1024,  # Set max number of new tokens
        num_beams=4,          # Keep beams, but consider reducing for diversity
        do_sample=False,       # Use sampling instead of beam search
        top_p=0.95,            # Use nucleus sampling for diversity
        early_stopping=False,  # Allow the model to generate more tokens
        top_k=50,
        temperature=1
    )
    result=tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [ ]:
# it is not working
def llm_flan(prompt, generate_params=None):
    # Specify to use CPU
    #device = torch.device("cpu")
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 1000),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [ ]:
llm_flan('how to lose belly fat')

In [ ]:
import torch
print(torch.cuda.is_available())


In [ ]:
# # alternative method creating with adding combined
# import lancedb
# from sentence_transformers import SentenceTransformer

# # Load a pre-trained model to generate text embeddings
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# # Example documents
# documents = [
#     {'id': 0, 'content': 'This is the first document.', 'number of sentences': 5, 'number of words': 56},
#     {'id': 1, 'content': 'This is the second document.', 'number of sentences': 5, 'number of words': 49}
# ]

# # Generate embeddings for the documents
# contents = [doc['content'] for doc in documents]
# embeddings = model.encode(contents)

# # Restructure the data
# data = [{"text": doc['content'], "embedding": embedding.tolist()} for doc, embedding in zip(documents, embeddings)]

# # Create or open a LanceDB database
# db = lancedb.connect("./lancedb")

# # Create the table and insert data
# table = db.create_table("my_vector_table", data=data)


In [ ]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation",
    model="google/gemma-7b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cpu",
)

messages = [
    {"role": "user", "content": "Who are you? Please, answer in pirate-speak."},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)
assistant_response = outputs[0]["generated_text"][-1]["content"]
print(assistant_response)


In [ ]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

model_path = 'openlm-research/open_llama_3b'
# model_path = 'openlm-research/open_llama_7b'

tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)

prompt = 'Q: What is the largest animal?\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=32
)
print(tokenizer.decode(generation_output[0]))


In [ ]:
# mistral

from huggingface_hub import HfApi, login
import os
api_token = os.getenv("huggingface")

# Log in with your Hugging Face API token
login(token=api_token)
# Authenticate Hugging Face API
#api = HfApi()
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the Mistral 7B tokenizer and model from Hugging Face
model_name = "mistralai/Mistral-7B-v0.1"  # The model's path on Hugging Face

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Define a function to generate text using the Mistral model
def generate_text(prompt, max_new_tokens=256):
    # Encode the prompt using the tokenizer
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate tokens
    outputs = model.generate(inputs.input_ids, max_new_tokens=max_new_tokens)
    
    # Decode the tokens to generate the text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

# Example usage
prompt = "Tell me about the benefits of healthy food."
generated_output = generate_text(prompt, max_new_tokens=150)
print(generated_output)


In [ ]:
import os
#from dotenv import load_dotenv

os.environ['RUN_TIMEZONE_CHECK'] = '0'

from db import init_db

load_dotenv()

if __name__ == "__main__":
    print("Initializing database...")
    init_db()